In [9]:
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split

# 설정 변수
random_state = 12
mlrsnet_root = '/home/son/ml/datasets/MLRSNet'      # MLRSNet 데이터셋의 루트 디렉토리 위치
output_dir = '/home/son/ml/xai_new/split_data_v2'   # CSV 파일 저장 경로 설정

labels_dir = os.path.join(mlrsnet_root, 'Labels')

label_files = glob.glob(os.path.join(labels_dir, '*.csv'))

all_labels_df = pd.DataFrame()

for file in label_files:
    df = pd.read_csv(file)
    class_name = os.path.splitext(os.path.basename(file))[0]
    df['image'] = df['image'].apply(lambda x: os.path.join('Images', class_name, x))
    all_labels_df = pd.concat([all_labels_df, df], ignore_index=True)

# 이미지 경로에 MLRSNet 루트 경로를 추가
all_labels_df['image'] = all_labels_df['image'].apply(lambda x: os.path.join(mlrsnet_root, x))

# 데이터 섞기
all_labels_df = all_labels_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

# 데이터 분할 비율 설정 (예: 70% train, 15% val, 15% test)
train_ratio = 0.4
val_ratio = 0.1
test_ratio = 0.5

train_df, temp_df = train_test_split(all_labels_df, test_size=(1 - train_ratio), random_state=random_state)

val_df, test_df = train_test_split(temp_df, test_size=(test_ratio / (test_ratio + val_ratio)), random_state=random_state)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# CSV 파일로 저장
train_df.to_csv(os.path.join(output_dir, 'train.csv'), index=False)
val_df.to_csv(os.path.join(output_dir, 'val.csv'), index=False)
test_df.to_csv(os.path.join(output_dir, 'test.csv'), index=False)

Train set size: 43664
Validation set size: 10916
Test set size: 54581
